# Libraries

In [107]:
import pandas as pd
from getpass import getuser
from collections import defaultdict

# Load and inspect dataset

In [108]:
# Get the current user's name
user = getuser()

# Path to the dataset
data_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\fifa.csv'

# Read the dataset with a different encoding
df = pd.read_csv(data_path, encoding='ISO-8859-1')

# Extract relevant columns

In [109]:
# First, let's extract the year from the 'tournament_id' column and create a new column 'year'
df['year'] = df['tournament_id'].str.extract(r'WC-(\d{4})').astype(int)

# Now, filter the data for tournaments after 1986
filtered_df = df[df['year'] > 1984]

# Extract relevant columns for goal events and match results
goals_df = filtered_df[['tournament_name', 'group_name','match_name', 'match_id', 'player_team_name','match_date', 'minute_regulation', 'minute_stoppage', 
               'team_id', 'own_goal']]

# Remove all observations where group_name is "not applicable"
goals_df = goals_df[goals_df['group_name'] != 'not applicable']

# Convert 'match_date' to datetime format
goals_df['match_date'] = pd.to_datetime(goals_df['match_date'], format='%m/%d/%Y')

# Sort the dataset by 'match_date' in ascending order (oldest first) and 'minute_regulation'
goals_df = goals_df.sort_values(by=['match_date', 'minute_regulation'], ascending=[True, True])

# Display the first few rows to confirm the sorting
goals_df.head()


,tournament_name,group_name,match_name,match_id,player_team_name,match_date,minute_regulation,minute_stoppage,team_id,own_goal
1196,1986 FIFA World Cup,Group A,Bulgaria v Italy,M-1986-01,Italy,1986-05-31,44,0,T-39,0
1197,1986 FIFA World Cup,Group A,Bulgaria v Italy,M-1986-01,Bulgaria,1986-05-31,85,0,T-10,0
1198,1986 FIFA World Cup,Group D,Spain v Brazil,M-1986-02,Brazil,1986-06-01,62,0,T-09,0
1199,1986 FIFA World Cup,Group C,Canada v France,M-1986-03,France,1986-06-01,79,0,T-28,0
1204,1986 FIFA World Cup,Group C,Soviet Union v Hungary,M-1986-05,Soviet Union,1986-06-02,2,0,T-70,0


# Recreate Leauge Table after first two matchdays

### Filter out the matches from the last match day in the goals_df DataFrame.

In [110]:
# Step 1: Find the last match date for each tournament and group
last_dates = goals_df.groupby(['tournament_name', 'group_name'])['match_date'].max().reset_index()

# Step 2: Create the dataset with all games excluding the last match day (goals_before_last_day)
goals_before_last_day = goals_df.merge(last_dates, on=['tournament_name', 'group_name', 'match_date'], how='outer', indicator=True)
goals_before_last_day = goals_before_last_day[goals_before_last_day['_merge'] == 'left_only'].drop(columns=['_merge'])

# Step 3: Create the dataset with only the last match day games (goals_last_day)
goals_last_day = goals_df.merge(last_dates, on=['tournament_name', 'group_name', 'match_date'])

# Step 4: Split match_name into home and away teams and remove match_name column for both datasets

# For goals_before_last_day
goals_before_last_day[['home', 'away']] = goals_before_last_day['match_name'].str.split(' v ', expand=True)
goals_before_last_day = goals_before_last_day.drop(columns=['match_name'])

# For goals_last_day
goals_last_day[['home', 'away']] = goals_last_day['match_name'].str.split(' v ', expand=True)
goals_last_day = goals_last_day.drop(columns=['match_name'])

# Sort the DataFrame by 'minute_regulation' and 'minute_stoppage'
goals_last_day_sorted = goals_last_day.sort_values(by=['minute_regulation', 'minute_stoppage'], ascending=[True, True])

## create df  of aggregate data for each match

In [111]:
# Initialize an empty list to store the results
results = []

# Iterate over each match in the dataset
for match_id, group in goals_before_last_day.groupby(['tournament_name', 'group_name', 'match_id', 'home', 'away']):
    # Initialize goals_home and goals_away for each match
    goals_home = 0
    goals_away = 0
    
    # Loop through each row in the group
    for _, row in group.iterrows():
        # Check if the team is the home team and no own goal
        if row['home'] == row['player_team_name'] and row['own_goal'] == 0:
            goals_home += 1
        # If the team is the away team and no own goal, increment goals_away
        elif row['away'] == row['player_team_name'] and row['own_goal'] == 0:
            goals_away += 1

    # Append the results to the list, keeping home and away columns
    results.append({
        'tournament_name': match_id[0],
        'group_name': match_id[1],
        'match_id': match_id[2],
        'home': match_id[3],
        'away': match_id[4],
        'goals_home': goals_home,
        'goals_away': goals_away
    })

# Convert the list into a DataFrame
agg_goals_before_last_day = pd.DataFrame(results)

# Add a new column 'won' based on the comparison of goals_home and goals_away
agg_goals_before_last_day['won'] = agg_goals_before_last_day.apply(
    lambda row: 1 if row['goals_home'] > row['goals_away'] else (-1 if row['goals_home'] < row['goals_away'] else 0), 
    axis=1
)

# Display the updated DataFrame (use print if outside of Jupyter)
display(agg_goals_before_last_day)



,tournament_name,group_name,match_id,home,away,goals_home,goals_away,won
0,1986 FIFA World Cup,Group A,M-1986-01,Bulgaria,Italy,1,1,0
1,1986 FIFA World Cup,Group A,M-1986-04,Argentina,South Korea,3,1,1
2,1986 FIFA World Cup,Group A,M-1986-13,Italy,Argentina,1,1,0
3,1986 FIFA World Cup,Group A,M-1986-15,South Korea,Bulgaria,1,1,0
4,1986 FIFA World Cup,Group B,M-1986-07,Belgium,Mexico,1,2,-1
...,...,...,...,...,...,...,...,...
266,2022 FIFA World Cup,Group G,M-2022-29,Cameroon,Serbia,3,3,0
267,2022 FIFA World Cup,Group G,M-2022-31,Brazil,Switzerland,1,0,1
268,2022 FIFA World Cup,Group H,M-2022-15,Portugal,Ghana,3,2,1
269,2022 FIFA World Cup,Group H,M-2022-30,South Korea,Ghana,2,3,-1


## aggregate data for home and away games

In [112]:
# Step 1: Home games aggregation with match count
home_games = agg_goals_before_last_day.groupby(['tournament_name', 'group_name', 'home']).agg(
    goals_scored=('goals_home', 'sum'),
    goals_conceded=('goals_away', 'sum'),
    points_home=('won', lambda x: sum(3 if result == 1 else (1 if result == 0 else 0) for result in x)),
    match_count_home=('match_id', 'count')  # Count how many match_id for each team at home
).reset_index()

# Step 2: Away games aggregation with match count
away_games = agg_goals_before_last_day.groupby(['tournament_name', 'group_name', 'away']).agg(
    goals_scored=('goals_away', 'sum'),
    goals_conceded=('goals_home', 'sum'),
    points_away=('won', lambda x: sum(3 if result == -1 else (1 if result == 0 else 0) for result in x)),
    match_count_away=('match_id', 'count')  # Count how many match_id for each team at away
).reset_index()


## aggregate data after first two matches

In [113]:
def uefa_before_last(home_games, away_games, agg_goals_before_last_day):
    # Step 1: Merge home_games and away_games on tournament_name, group_name, and home with away
    all_games_before_last = pd.merge(
        home_games,
        away_games,
        left_on=['tournament_name', 'group_name', 'home'],
        right_on=['tournament_name', 'group_name', 'away'],
        how='outer',
        suffixes=('_home', '_away')
    )

    # Step 2: For teams that only appear in home_games, assign home to team and copy relevant columns
    all_games_before_last['team'] = all_games_before_last['home'].fillna(all_games_before_last['away'])

    # Step 3: For goals_scored, goals_conceded, and points, handle missing values
    all_games_before_last['goals_scored'] = all_games_before_last['goals_scored_home'].fillna(0) + all_games_before_last['goals_scored_away'].fillna(0)
    all_games_before_last['goals_conceded'] = all_games_before_last['goals_conceded_home'].fillna(0) + all_games_before_last['goals_conceded_away'].fillna(0)
    all_games_before_last['points'] = all_games_before_last['points_home'].fillna(0) + all_games_before_last['points_away'].fillna(0)

    # Step 4: Add goals_difference column
    all_games_before_last['goals_difference'] = all_games_before_last['goals_scored'] - all_games_before_last['goals_conceded']

    # Step 5: Sum match_count_home and match_count_away to get total match count for each team
    all_games_before_last['total_matches'] = all_games_before_last['match_count_home'].fillna(0) + all_games_before_last['match_count_away'].fillna(0)

    # Step 6: Adjust points if total_matches is 1 (indicating a likely 0-0 draw)
    all_games_before_last.loc[all_games_before_last['total_matches'] == 1, 'points'] += 1

    # Step 7: Drop unnecessary columns used in the merge process
    all_games_before_last = all_games_before_last[['tournament_name', 'group_name', 'team', 'goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']]

    # Step 8: Initial sort by points, goals_difference, and goals_scored within each tournament and group
    all_games_before_last = all_games_before_last.sort_values(
        by=['tournament_name', 'group_name', 'points', 'goals_difference', 'goals_scored'],
        ascending=[True, True, False, False, False]
    ).reset_index(drop=True)

    # Step 9: Initialize a new column for tie-break results, which will store either the tie-break result or 'no need'
    all_games_before_last['tiebreaker'] = 'no need'

    # Step 10: Define the function to check the tie-breaker
    def check_tiebreaker(row1, row2, agg_data):
        """
        Check the tie-breaker based on the head-to-head match result from agg_goals_before_last_day.
        Return the team that won (if any) or 'tie'.
        """
        # Look for the head-to-head match between the two teams in agg_goals_before_last_day
        match = agg_data[((agg_data['home'] == row1['team']) & (agg_data['away'] == row2['team'])) |
                         ((agg_data['home'] == row2['team']) & (agg_data['away'] == row1['team']))]
        
        if not match.empty:
            if match.iloc[0]['won'] == 1:
                return row1['team']  # Home team won
            elif match.iloc[0]['won'] == -1:
                return row2['team']  # Away team won
            else:
                return 'tie'  # It's a draw
        else:
            return 'no result'  # No match found

    # Step 11: Loop through the sorted dataframe and apply the tie-breaker if needed
    for i in range(len(all_games_before_last) - 1):
        row1 = all_games_before_last.iloc[i]
        row2 = all_games_before_last.iloc[i + 1]

        # Check if the two rows have identical values for points
        if row1['points'] == row2['points']:
            print(f"Tie in {row1['tournament_name']}, Group: {row1['group_name']}, between {row1['team']} and {row2['team']} (Points: {row1['points']})")

            # Apply the tie-breaker by checking the head-to-head match result
            tiebreak_result = check_tiebreaker(row1, row2, agg_goals_before_last_day)

            if tiebreak_result != 'tie' and tiebreak_result != 'no result':
                print(f"Tie resolved by head-to-head: {tiebreak_result} won in {row1['tournament_name']}, Group: {row1['group_name']}")
                all_games_before_last.at[i, 'tiebreaker'] = tiebreak_result
                all_games_before_last.at[i + 1, 'tiebreaker'] = tiebreak_result
            else:
                # If still tied after the tiebreaker, check goals_difference and goals_scored
                if row1['goals_difference'] > row2['goals_difference']:
                    print(f"Tie resolved by goals difference: {row1['team']} favored in {row1['tournament_name']}, Group: {row1['group_name']}")
                    all_games_before_last.at[i, 'tiebreaker'] = row1['team']
                    all_games_before_last.at[i + 1, 'tiebreaker'] = row1['team']
                elif row1['goals_difference'] < row2['goals_difference']:
                    print(f"Tie resolved by goals difference: {row2['team']} favored in {row1['tournament_name']}, Group: {row1['group_name']}")
                    all_games_before_last.at[i, 'tiebreaker'] = row2['team']
                    all_games_before_last.at[i + 1, 'tiebreaker'] = row2['team']
                elif row1['goals_scored'] > row2['goals_scored']:
                    print(f"Tie resolved by goals scored: {row1['team']} favored in {row1['tournament_name']}, Group: {row1['group_name']}")
                    all_games_before_last.at[i, 'tiebreaker'] = row1['team']
                    all_games_before_last.at[i + 1, 'tiebreaker'] = row1['team']
                else:
                    print(f"Tie resolved by goals scored: {row2['team']} favored in {row1['tournament_name']}, Group: {row1['group_name']}")
                    all_games_before_last.at[i, 'tiebreaker'] = row2['team']
                    all_games_before_last.at[i + 1, 'tiebreaker'] = row2['team']

    # Step 12: Add group standing by ranking teams within each group based on points, goal difference, and goals scored
    all_games_before_last['standing'] = all_games_before_last.groupby(['tournament_name', 'group_name']).apply(
        lambda x: x.sort_values(by=['points', 'goals_difference', 'goals_scored'], ascending=[False, False, False])
    ).reset_index(drop=True).groupby(['tournament_name', 'group_name']).cumcount() + 1

    # Convert goals_scored, goals_conceded, points, goals_difference, and total_matches to integers
    all_games_before_last[['goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']] = all_games_before_last[
        ['goals_scored', 'goals_conceded', 'points', 'goals_difference', 'total_matches']].astype(int)

    return all_games_before_last


In [114]:
all_games_before_last = uefa_before_last(home_games, away_games, agg_goals_before_last_day)
display(all_games_before_last)

Tie in 1986 FIFA World Cup, Group: Group A, between Bulgaria and Italy (Points: 2.0)
Tie resolved by goals scored: Italy favored in 1986 FIFA World Cup, Group: Group A
Tie in 1986 FIFA World Cup, Group: Group B, between Mexico and Paraguay (Points: 4.0)
Tie resolved by goals scored: Mexico favored in 1986 FIFA World Cup, Group: Group B
Tie in 1986 FIFA World Cup, Group: Group C, between Soviet Union and France (Points: 4.0)
Tie resolved by goals difference: Soviet Union favored in 1986 FIFA World Cup, Group: Group C
Tie in 1986 FIFA World Cup, Group: Group D, between Northern Ireland and Algeria (Points: 1.0)
Tie resolved by goals scored: Northern Ireland favored in 1986 FIFA World Cup, Group: Group D
Tie in 1990 FIFA World Cup, Group: Group A, between Czechoslovakia and Italy (Points: 6.0)
Tie resolved by goals difference: Czechoslovakia favored in 1990 FIFA World Cup, Group: Group A
Tie in 1990 FIFA World Cup, Group: Group A, between Austria and United States (Points: 0.0)
Tie resolv

C:\Users\aldi\AppData\Local\Temp\ipykernel_2244\2976437292.py:97: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_games_before_last['standing'] = all_games_before_last.groupby(['tournament_name', 'group_name']).apply(


,tournament_name,group_name,team,goals_scored,goals_conceded,points,goals_difference,total_matches,tiebreaker,standing
0,1986 FIFA World Cup,Group A,Argentina,4,2,4,2,2,no need,1
1,1986 FIFA World Cup,Group A,Bulgaria,2,2,2,0,2,Italy,2
2,1986 FIFA World Cup,Group A,Italy,2,2,2,0,2,Italy,3
3,1986 FIFA World Cup,Group A,South Korea,2,4,1,-2,2,no need,4
4,1986 FIFA World Cup,Group B,Mexico,3,2,4,1,2,Mexico,1
...,...,...,...,...,...,...,...,...,...,...
289,2022 FIFA World Cup,Group G,Serbia,3,5,1,-2,2,Cameroon,4
290,2022 FIFA World Cup,Group H,Portugal,5,2,6,3,2,no need,1
291,2022 FIFA World Cup,Group H,Ghana,5,5,3,0,2,no need,2
292,2022 FIFA World Cup,Group H,South Korea,2,3,1,-1,1,South Korea,3


# Recreate league table after last match day

In [115]:
def uefa_final_wc(tournament_name, group_name, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day):
    # Step 1: Filter the data for the specific tournament_name and group_name
    group_goals_tracking = all_games_before_last[
        (all_games_before_last['tournament_name'] == tournament_name) & 
        (all_games_before_last['group_name'] == group_name)
    ].copy()

    group_goals_last_day = goals_last_day_sorted[
        (goals_last_day_sorted['tournament_name'] == tournament_name) & 
        (goals_last_day_sorted['group_name'] == group_name)
    ]

    # Step 2: Initialize columns for tracking team performance
    group_goals_tracking['before_last_game_goals_scored'] = group_goals_tracking['goals_scored']
    group_goals_tracking['before_last_game_goals_conceded'] = group_goals_tracking['goals_conceded']
    group_goals_tracking['before_last_game_standing'] = group_goals_tracking['standing']
    group_goals_tracking['before_last_game_points'] = group_goals_tracking['points']

    group_goals_tracking['last_game_goals_scored'] = 0
    group_goals_tracking['last_game_goals_conceded'] = 0
    group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored']
    group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded']
    group_goals_tracking['total_goal_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']
    group_goals_tracking['last_game_points'] = 0
    group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points']
    group_goals_tracking['last_game_standing'] = 0
    group_goals_tracking['1st'] = 0
    group_goals_tracking['2nd'] = 0
    group_goals_tracking['3rd'] = 0
    group_goals_tracking['4th'] = 0
    group_goals_tracking['changes'] = 0  # Initialize this but will be redefined later as the sum of 1st, 2nd, 3rd, 4th
    group_goals_tracking['tied'] = False  # Initialize a flag to track tied teams
    group_goals_tracking['tied_won'] = 0  # Initialize to track if the team won a tie-breaker match

    # Step 3: Sort group_goals_last_day by minute_regulation and minute_stoppage in ascending order
    group_goals_last_day = group_goals_last_day.sort_values(by=['minute_regulation', 'minute_stoppage'], ascending=[True, True])

    # Step 4: Iterate through the sorted and filtered last match goals and update the goals_tracking table
    previous_standings = group_goals_tracking['last_game_standing'].copy()
    first_iteration = True  # Variable to track the first iteration

    for _, goal in group_goals_last_day.iterrows():
        home_team = goal['home']
        away_team = goal['away']
        player_team = goal['player_team_name']
        own_goal = goal['own_goal']

        # Print goal information for each goal
        print(f"Analyzing goal: {goal['minute_regulation']} minute, {goal['minute_stoppage']} extra time, Player team: {player_team}, Home: {home_team}, Away: {away_team}")

        # Update the goals based on who scored the goal
        if own_goal == 0:  # Normal goal
            if player_team == home_team:
                # Home team scored, update home scored and away conceded
                group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_scored'] += 1
                group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_conceded'] += 1
            elif player_team == away_team:
                # Away team scored, update away scored and home conceded
                group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_scored'] += 1
                group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_conceded'] += 1
        elif own_goal == 1:  # Own goal
            if player_team == home_team:
                # Own goal by home team, away team scores
                group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_scored'] += 1
                group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_conceded'] += 1
            elif player_team == away_team:
                # Own goal by away team, home team scores
                group_goals_tracking.loc[group_goals_tracking['team'] == home_team, 'last_game_goals_scored'] += 1
                group_goals_tracking.loc[group_goals_tracking['team'] == away_team, 'last_game_goals_conceded'] += 1

        # Step 5: Update total_goals_scored, total_goals_conceded, and total_goal_difference
        group_goals_tracking['total_goals_scored'] = group_goals_tracking['before_last_game_goals_scored'] + group_goals_tracking['last_game_goals_scored']
        group_goals_tracking['total_goals_conceded'] = group_goals_tracking['before_last_game_goals_conceded'] + group_goals_tracking['last_game_goals_conceded']
        group_goals_tracking['total_goal_difference'] = group_goals_tracking['total_goals_scored'] - group_goals_tracking['total_goals_conceded']

        # Step 6: Assign points for the last game dynamically after each goal is processed
        for i, row in group_goals_tracking.iterrows():
            if row['last_game_goals_scored'] > row['last_game_goals_conceded']:
                group_goals_tracking.loc[i, 'last_game_points'] = 3  # Win
            elif row['last_game_goals_scored'] == row['last_game_goals_conceded']:
                group_goals_tracking.loc[i, 'last_game_points'] = 1  # Draw
            else:
                group_goals_tracking.loc[i, 'last_game_points'] = 0  # Loss

        # Step 7: Update total points
        group_goals_tracking['total_points'] = group_goals_tracking['before_last_game_points'] + group_goals_tracking['last_game_points']

        # Step 8: Mark teams that are tied
        group_goals_tracking['tied'] = group_goals_tracking.duplicated(subset=['total_points'], keep=False)

        # Reset `tied_won` to 0 for all teams
        group_goals_tracking['tied_won'] = 0

        # Step 8b: Calculate tied_won only for tied teams
        tied_teams = group_goals_tracking[group_goals_tracking['tied']]

        if not tied_teams.empty:
            # Iterate over tied teams to resolve standings using head-to-head results
            for index, row in tied_teams.iterrows():
                team1 = row['team']
                team1_index = index

                # Look for other teams tied with this team
                for other_index, other_row in tied_teams[tied_teams.index != index].iterrows():
                    team2 = other_row['team']
                    team2_index = other_index

                    # Check if these two teams played against each other in agg_goals_before_last_day
                    match = agg_goals_before_last_day[
                        ((agg_goals_before_last_day['home'] == team1) & (agg_goals_before_last_day['away'] == team2)) |
                        ((agg_goals_before_last_day['home'] == team2) & (agg_goals_before_last_day['away'] == team1))
                    ]

                    if not match.empty:
                        match_result = match.iloc[0]['won']  # Assume the 'won' column holds 1 for home win, -1 for away win, 0 for draw

                        # Resolve the tie using the match result
                        if match_result == 1:
                            # Home team won
                            if match.iloc[0]['home'] == team1:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tied_won'] = 1
                            else:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tied_won'] = 1
                        elif match_result == -1:
                            # Away team won
                            if match.iloc[0]['away'] == team1:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tied_won'] = 1
                            else:
                                group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tied_won'] = 1
                        elif match_result == 0:
                            # Draw, both teams get the same standing
                            group_goals_tracking.loc[group_goals_tracking['team'] == team1, 'tied_won'] = 0
                            group_goals_tracking.loc[group_goals_tracking['team'] == team2, 'tied_won'] = 0

        # Step 8c: Sort teams by total points, tied_won, goal difference, and goals scored
        group_goals_tracking = group_goals_tracking.sort_values(by=['total_points', 'tied_won', 'total_goal_difference', 'total_goals_scored'],
                                                                ascending=[False, False, False, False])

        # Step 9: Assign standings based on the sorting and tie resolution
        group_goals_tracking['last_game_standing'] = group_goals_tracking.reset_index(drop=True).index + 1

        # Step 10: Track changes and update standing positions after each goal is processed
        for i, row in group_goals_tracking.iterrows():
            team = row['team']

            if first_iteration:
                # Skip the update if the standing did not change
                if row['before_last_game_standing'] == row['last_game_standing']:
                    continue  # Skip updating the counters
                else:
                    # Update the position counters since standing has changed
                    if row['last_game_standing'] == 1:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '1st'] += 1
                    elif row['last_game_standing'] == 2:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '2nd'] += 1
                    elif row['last_game_standing'] == 3:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '3rd'] += 1
                    elif row['last_game_standing'] == 4:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '4th'] += 1

                # Disable first iteration flag after the first goal
                first_iteration = False
            else:
                # Track the standing position counters only if the position differs from the previous state
                if row['last_game_standing'] != previous_standings[i]:  
                    if row['last_game_standing'] == 1:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '1st'] += 1
                    elif row['last_game_standing'] == 2:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '2nd'] += 1
                    elif row['last_game_standing'] == 3:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '3rd'] += 1
                    elif row['last_game_standing'] == 4:
                        group_goals_tracking.loc[group_goals_tracking['team'] == team, '4th'] += 1

        # Update previous standings after each goal
        previous_standings = group_goals_tracking['last_game_standing'].copy()

        # Step 11: Calculate changes as the sum of 1st, 2nd, 3rd, and 4th
        group_goals_tracking['changes'] = group_goals_tracking[['1st', '2nd', '3rd', '4th']].sum(axis=1)

        # Step 12: Print the updated group_goals_tracking after processing each goal
        print("\n=== Updated Standings After This Goal ===\n")
        display_columns = ['team', 'total_points', 'total_goals_scored', 'total_goals_conceded', 
                           'total_goal_difference', 'last_game_points', 'last_game_standing', 
                           'changes', '1st', '2nd', '3rd', '4th', 'tied', 'tied_won']
        print(group_goals_tracking[display_columns].to_string(index=False))
        print("\n========================================\n")

    # Step 13: Return the final DataFrame
    return group_goals_tracking


In [116]:
# Initialize an empty list to store the results for each pair
all_results = []

# Get unique pairs of tournament_name and group_name
unique_pairs = all_games_before_last[['tournament_name', 'group_name']].drop_duplicates()

# Loop through each unique pair
for _, row in unique_pairs.iterrows():
    tournament_name = row['tournament_name']
    group_name = row['group_name']
    
    # Apply the function to the current pair
    result = uefa_final_wc(tournament_name, group_name, all_games_before_last, goals_last_day_sorted, agg_goals_before_last_day)
    
    # Append the result to the list
    all_results.append(result)

# Concatenate all the results into a single DataFrame
changes_df_wc = pd.concat(all_results)

# Keep only the specified columns
changes_df_wc = changes_df_wc[['tournament_name', 'group_name', 'team', '1st', '2nd', '3rd', '4th', 'changes']]

# Display the final DataFrame
display(changes_df_wc)


Analyzing goal: 4 minute, 0 extra time, Player team: Argentina, Home: Argentina, Away: Bulgaria

=== Updated Standings After This Goal ===

       team  total_points  total_goals_scored  total_goals_conceded  total_goal_difference  last_game_points  last_game_standing  changes  1st  2nd  3rd  4th  tied  tied_won
  Argentina             7                   5                     2                      3                 3                   1        0    0    0    0    0 False         0
      Italy             3                   2                     2                      0                 1                   2        1    0    1    0    0 False         0
   Bulgaria             2                   2                     3                     -1                 0                   3        1    0    0    1    0  True         0
South Korea             2                   2                     4                     -2                 1                   4        1    0    0    0    1  True 

,tournament_name,group_name,team,1st,2nd,3rd,4th,changes
0,1986 FIFA World Cup,Group A,Argentina,0,0,0,0,0
2,1986 FIFA World Cup,Group A,Italy,0,1,0,0,1
1,1986 FIFA World Cup,Group A,Bulgaria,0,0,1,0,1
3,1986 FIFA World Cup,Group A,South Korea,0,0,0,1,1
4,1986 FIFA World Cup,Group B,Mexico,1,1,0,0,2
...,...,...,...,...,...,...,...,...
289,2022 FIFA World Cup,Group G,Serbia,0,1,0,1,2
290,2022 FIFA World Cup,Group H,Portugal,0,0,0,0,0
292,2022 FIFA World Cup,Group H,South Korea,0,1,0,1,2
293,2022 FIFA World Cup,Group H,Uruguay,0,1,2,0,3


In [117]:
# Exporting final df
file_path = rf'C:\Users\{user}\Documents\GitHub\tiebreak_wc\data\tb_wc_uefa.xlsx'
changes_df_wc.to_excel(file_path, index=False)